In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

# from sklearn.pipeline import make_pipeline
# model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())
from nba_api.stats.endpoints import leaguedashteamstats

teams_dict, teams_list = get_teams(league="NBA")
box_DIR = "../data/box/"

In [ ]:
year = 2021
season_str = str(year) + "-" + str(year + 1)[-2:]

In [ ]:
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="T",
    season=year,
    season_type_all_star="Regular Season",
)
df1 = stats.get_data_frames()[0]
df1["HOME"] = ~df1["MATCHUP"].str.contains("@")
cols = [
    'TEAM_ID',  'TEAM_NAME', 'GAME_ID',
    'GAME_DATE', 'MATCHUP', 'WL','HOME'
]
df1 = df1[cols]

In [ ]:
df2 = df1.groupby("GAME_ID")
df2_1 = df2.nth(0)
df2_2 = df2.nth(1)
df2_3 = pd.merge(df2_1, df2_2, on="GAME_ID",suffixes=["","_1"])
df2_4 = df2.nth(1)
df2_5 = df2.nth(0)
df2_6 = pd.merge(df2_4, df2_5, on="GAME_ID",suffixes=["","_1"])
df3 = pd.concat([df2_3, df2_6]).sort_values(by="GAME_ID").reset_index(drop=True)
df4 = df3.query("TEAM_ID == 1610612755").sort_values("GAME_DATE").reset_index(drop=True)

In [ ]:
stats = leaguedashteamstats.LeagueDashTeamStats(
    per_mode_detailed="PerGame",
    season_type_all_star="Regular Season",
    season=season_str
)
df51 = stats.get_data_frames()[0]
df5 = df51.query("W_PCT > 0.500")
t_500 = list(df5["TEAM_ID"].unique())

In [ ]:
df6 = df4[df4["TEAM_ID_1"].isin(t_500)].reset_index(drop=True)
df7 = df4[~df4["TEAM_ID_1"].isin(t_500)].reset_index(drop=True)

In [ ]:
stats = leaguegamelog.LeagueGameLog(
    player_or_team_abbreviation="P",
    season=year,
    season_type_all_star="Regular Season",
)
df81 = stats.get_data_frames()[0]
df8 = df81.query("PLAYER_ID  == 203954").reset_index(drop=True)
df8 = df8[["GAME_ID","TEAM_ID","PLAYER_NAME"]]

In [ ]:
df9 = pd.merge(df6,df8,on=["GAME_ID","TEAM_ID"],how="left")
df9["Status"] = np.where(df9["PLAYER_NAME"] == "Joel Embiid","Played","Missed")
df10 = pd.merge(df7,df8,on=["GAME_ID","TEAM_ID"],how="left")
df10["Status"] = np.where(df10["PLAYER_NAME"] == "Joel Embiid","Played","Missed")
df91 = df9[~df9["HOME"]]
df92 = df10[~df10["HOME"]]
cols = ['GAME_DATE', 'MATCHUP','Status']
dft1 = df91[cols].reset_index(drop=True)
dft2 = df92[cols].reset_index(drop=True)

In [ ]:
%reload_ext rpy2.ipython

In [ ]:
%%R -i dft1
df <- dft1
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Joel Embiid Games vs Teams over 0.500 on Road**"),
    subtitle = md("**2021-22 NBA Season**"),
    ) %>%
    cols_label( GAME_DATE="Date", MATCHUP="Matchup"
    ) %>%
    data_color(columns = "Status", palette = c("violet","green")) %>%
    cols_align(align = "center") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 10,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 8,
        heading.title.font.size = 12,
        heading.subtitle.font.size = 11,
        table.font.names = "Consolas", 
        data_row.padding = px(0.5),
        # table.margin.left = 1,
        # table.margin.right = 1,

    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/joel_road_O500.png",zoom=5) 

In [ ]:
%%R -i dft2
df <- dft2
library(tidyverse)
library(gt)
library(gtExtras)
df %>% 
  gt()%>%
  tab_header(
    title = md("**Joel Embiid Games vs Teams under 0.500 on Road**"),
    subtitle = md("**2021-22 NBA Season**"),
    ) %>%
    cols_label( GAME_DATE="Date", MATCHUP="Matchup"
    ) %>%
    data_color(columns = "Status", palette = c("violet","green")) %>%
    cols_align(align = "center") %>%
    tab_options(
        table.background.color = "floralwhite",
        column_labels.font.size = 10,
        column_labels.font.weight = 'bold',
        row_group.font.weight = 'bold',
        row_group.background.color = "#E5E1D8",
        table.font.size = 8,
        heading.title.font.size = 12,
        heading.subtitle.font.size = 11,
        table.font.names = "Consolas", 
        data_row.padding = px(0.5),
        # table.margin.left = 1,
        # table.margin.right = 1,

    ) %>% 
    tab_source_note(
    source_note = "@SravanNBA | Source: nba.com/stats" ) %>% gtsave("../figs/team_ratings/joel_road_U500.png",zoom=5) 